Adapted from https://www.tutorialspoint.com/webgl/webgl_sample_application.htm

In [1]:
import feedWebGL2.feedback as fd
from ipywidgets import interact, interactive, fixed, interact_manual
fd.widen_notebook()

In [2]:

vertex_shader = """#version 300 es
    uniform float scale;
    in vec3 coordinates;
    in vec3 offset;
    in vec3 color;
    out vec3 output_vertex;
    out vec3 output_color;

    void main() {
        vec3 scaled = scale * (coordinates + offset);
        gl_Position = vec4(scaled, 1.0);
        output_vertex = gl_Position.xyz;
        output_color = color;
    }
"""

fragment_shader = """#version 300 es
    precision highp float;
    in vec3 output_color;
    out vec4 color;

    void main() {
        color = vec4(output_color, 1.0);
    }
"""

vertices = [-1, 0, 0, -1, -1, 0, 0, -1, 0]
offsets = [0, 0, 0, 0, 1, 0.1, 1, 0, 0.2, 1, 1, 0.3]
colors = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1]

feedback_program = fd.FeedbackProgram(
    context = fd.Context(
        buffers = fd.Buffers(
            coordinates_buffer = fd.Buffer(array=vertices),
            offsets_buffer = fd.Buffer(array=offsets),
            colors_buffer = fd.Buffer(array=colors)
        ),
        width = 600,
        show = True,
    ),
    runner = fd.Runner(
        num_instances = 4,
        vertices_per_instance = 3,
        run_type = "TRIANGLES",
        uniforms = fd.Uniforms(
            scale = fd.Uniform(
                default_value = [0.5],
            ),
        ),
        inputs = fd.Inputs(
            coordinates = fd.Input(
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "coordinates_buffer",
                )
            ),
            color = fd.Input(
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "colors_buffer",
                )
            ),
            offset = fd.Input(
                per_vertex = False,
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "offsets_buffer",
                )
            ),
        ),
    ),
    program = fd.Program(
        vertex_shader = vertex_shader,
        fragment_shader = fragment_shader,
        feedbacks = fd.Feedbacks(
            output_vertex = fd.Feedback(num_components=3),
        ),
    ),
)

# display the widget and debugging information
#feedback_program.debugging_display()
feedback_program.run()
feedback_program

FeedbackProgram(status='deferring flush until render')

In [3]:
def move_corner(x=-0.5, scale=0.5, offsetx=0.0):
    new_vertices = vertices[:]
    new_vertices[0] = x
    feedback_program.change_buffer("coordinates_buffer", new_vertices)
    new_offsets = offsets[:]
    new_offsets[0] = offsetx
    feedback_program.change_buffer("offsets_buffer", new_offsets)
    feedback_program.change_uniform_vector("scale", [scale])
    feedback_program.run()

interact(move_corner, x=(-1.0, 1.0), scale=(0.0, 1.0))

interactive(children=(FloatSlider(value=-0.5, description='x', max=1.0, min=-1.0), FloatSlider(value=0.5, desc…

<function __main__.move_corner(x=-0.5, scale=0.5, offsetx=0.0)>

In [4]:
move_corner(x=-0.1)

In [5]:
feedback_program.get_feedback("output_vertex")

[[-0.05000000074505806, 0, 0],
 [-0.5, -0.5, 0],
 [0, -0.5, 0],
 [-0.05000000074505806, 0.5, 0.05000000074505806],
 [-0.5, 0, 0.05000000074505806],
 [0, 0, 0.05000000074505806],
 [0.44999998807907104, 0, 0.10000000149011612],
 [0, -0.5, 0.10000000149011612],
 [0.5, -0.5, 0.10000000149011612],
 [0.44999998807907104, 0.5, 0.15000000596046448],
 [0, 0, 0.15000000596046448],
 [0.5, 0, 0.15000000596046448]]

In [6]:
#import time
def ttest():
    for i in range(-100, 100):
        move_corner(i/100.0)
        #time.sleep(0.01)
        #print(i)
ttest()